In [1]:
#ete kernel
import pandas as pd
import matplotlib.pyplot as plt
import difflib
import os
import re
from ete3 import ClusterTree, TreeStyle, ProfileFace, Tree, TextFace
from ete3.treeview.faces import add_face_to_node
import loess
from loess import loess_1d
from collections import defaultdict 
import numpy as np
import pickle

import math

# os.chdir("../activity_heatmaps/gene_tree_heatmaps/")

# Prepare Data

In [2]:
# Read in activity data
seq_data = pd.read_pickle('../data/FullOrthologDF_20240930')
seq_data

,SpeciesName,Seq,Length,WxxLF_loc,SmoothedActivites,LinearCharge,LinearHydrophobicityKD,SmoothedActivitesLoess
0,Sordariomycetes_jgi|Acral2|2019554|gm1.4974_g,MALRIEVYNRIESSTASTALQRQDLRYTFRSNARAASGQANANYQA...,2928,1006,"[50849.75653537431, 50849.75653537431, 50849.7...","[0.0, 0.0, 0.2, 0.0, 0.0, 0.0, -0.2, 0.0, 0.2,...","[0.0, 0.0, 0.6666666666666666, 0.5466666666666...","[34049.34344014826, 34049.34344014826, 34049.3..."
1,Sordariomycetes_jgi|Acral2|2027520|fgenesh1_pg...,MWLVVRAGPSPLLQDLAARCHIDGMSMPLLHFDPPDFPLTGVALGI...,455,327,"[50311.68363475637, 50311.68363475637, 50311.6...","[0.0, 0.0, 0.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.0, ...","[0.0, 0.0, 0.7933333333333333, 0.6511111111111...","[58366.833469168625, 58366.833469168625, 58366..."
2,Sordariomycetes_jgi|Acral2|2034848|fgenesh1_kg...,MSITELDDFTGFEGGASTAYSSPGAPAVFDLPGASNHVGTISPQDL...,222,94,"[64062.654182288825, 64062.654182288825, 75414...","[0.0, 0.0, -0.2, -0.2, -0.4, -0.6, -0.6, -0.4,...","[0.0, 0.0, 0.5311111111111111, 0.5733333333333...","[36846.26635132608, 36846.26635132608, 36846.2..."
3,Sordariomycetes_jgi|Acral2|2034849|fgenesh1_kg...,MSITGNYNQHFGAAGIISSHNYVLSELDDFTGFEGGASTAYSSPGA...,243,115,"[103421.45065118768, 103421.45065118768, 10342...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.6, 0.48, 0.4688888888888889, 0.29...","[98417.43715391349, 98417.43715391349, 98417.4..."
5,Sordariomycetes_jgi|Acral2|2047914|estExt_Gene...,MADTCGGSTPLKNFSQYGSQDRSLQQDRVVHGFHGSAAAGPSTFRS...,2943,1021,"[49761.33963354764, 49761.33963354764, 49761.3...","[0.0, 0.0, -0.2, -0.2, -0.2, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.5444444444444445, 0.4933333333333...","[36959.46539862907, 36959.46539862907, 36959.4..."
...,...,...,...,...,...,...,...,...
1207,Cimm_XP_012214147.1_CoccidioidesImmitisRS,MSTSNLPLDIGTLLDLSTDQFVEDLGSSSHSSLLDQDQLDQLINFN...,242,108,"[179044.502422884, 179044.502422884, 179044.50...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2, -0.2, -0....","[0.0, 0.0, 0.4133333333333334, 0.4555555555555...","[215199.18805174268, 215199.18805174268, 21519..."
1208,Cpos_XP_003070205.1_CoccidioidesPosadasiiC735,MSTSNLPLGMVSLSASAVRLVANQRPDIGTLLDLSTDQYVEDLGSS...,260,126,"[56309.23879651313, 56309.23879651313, 56309.2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.4133333333333334, 0.4555555555555...","[6138.0, 6138.0, 6138.0, 6138.0, 6138.0, 6138...."
1209,Mory_EHA48851.1_MagnaportheOryzae70_15,MNNTSDLGLDDFTAFGGGASAFPSPAMPGVFDIASTTASTMGTVSP...,239,101,"[165405.40017749005, 165405.40017749005, 16540...","[0.0, 0.0, 0.0, -0.2, -0.2, -0.2, -0.2, -0.4, ...","[0.0, 0.0, 0.3533333333333334, 0.2333333333333...","[156903.4915015713, 156903.4915015713, 156903...."
1210,Nfis_EAW24893.1_NeosartoryaFischeriNRRL181,MSTPNIAQDMPDFFGLPSNDFGDDFELSTEPTMLSPNQIPTGLMAV...,251,109,"[148604.20780984958, 148604.20780984958, 14860...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2, -0.2, -0....","[0.0, 0.0, 0.39555555555555555, 0.453333333333...","[177960.75802414527, 177960.75802414527, 17796..."


In [3]:
# Function to find the location of the WxxLF motif
def find_WLF(s):
    p = re.compile("W..LF")
    for m in p.finditer(s):
        return m.start()
    return -1

In [4]:
# Get location of all WxxLF motifs
seq_data.loc[:, "Location_WxxLF"] = seq_data["Seq"].apply(find_WLF)

# Getting the WxxLF motif that is the furthest into a sequence, all other sequence will be aligned to this
align_to = max(seq_data["Location_WxxLF"])
align_to

1413

In [5]:
# Calculate padding i.e. how much "sequence" needs to be added to the front to align all the WxxLF motifs
seq_data.loc[:,"pad_by"] = list(seq_data["Location_WxxLF"] * -1 + align_to)
seq_data

,SpeciesName,Seq,Length,WxxLF_loc,SmoothedActivites,LinearCharge,LinearHydrophobicityKD,SmoothedActivitesLoess,Location_WxxLF,pad_by
0,Sordariomycetes_jgi|Acral2|2019554|gm1.4974_g,MALRIEVYNRIESSTASTALQRQDLRYTFRSNARAASGQANANYQA...,2928,1006,"[50849.75653537431, 50849.75653537431, 50849.7...","[0.0, 0.0, 0.2, 0.0, 0.0, 0.0, -0.2, 0.0, 0.2,...","[0.0, 0.0, 0.6666666666666666, 0.5466666666666...","[34049.34344014826, 34049.34344014826, 34049.3...",1006,407
1,Sordariomycetes_jgi|Acral2|2027520|fgenesh1_pg...,MWLVVRAGPSPLLQDLAARCHIDGMSMPLLHFDPPDFPLTGVALGI...,455,327,"[50311.68363475637, 50311.68363475637, 50311.6...","[0.0, 0.0, 0.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.0, ...","[0.0, 0.0, 0.7933333333333333, 0.6511111111111...","[58366.833469168625, 58366.833469168625, 58366...",327,1086
2,Sordariomycetes_jgi|Acral2|2034848|fgenesh1_kg...,MSITELDDFTGFEGGASTAYSSPGAPAVFDLPGASNHVGTISPQDL...,222,94,"[64062.654182288825, 64062.654182288825, 75414...","[0.0, 0.0, -0.2, -0.2, -0.4, -0.6, -0.6, -0.4,...","[0.0, 0.0, 0.5311111111111111, 0.5733333333333...","[36846.26635132608, 36846.26635132608, 36846.2...",94,1319
3,Sordariomycetes_jgi|Acral2|2034849|fgenesh1_kg...,MSITGNYNQHFGAAGIISSHNYVLSELDDFTGFEGGASTAYSSPGA...,243,115,"[103421.45065118768, 103421.45065118768, 10342...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.6, 0.48, 0.4688888888888889, 0.29...","[98417.43715391349, 98417.43715391349, 98417.4...",115,1298
5,Sordariomycetes_jgi|Acral2|2047914|estExt_Gene...,MADTCGGSTPLKNFSQYGSQDRSLQQDRVVHGFHGSAAAGPSTFRS...,2943,1021,"[49761.33963354764, 49761.33963354764, 49761.3...","[0.0, 0.0, -0.2, -0.2, -0.2, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.5444444444444445, 0.4933333333333...","[36959.46539862907, 36959.46539862907, 36959.4...",1021,392
...,...,...,...,...,...,...,...,...,...,...
1207,Cimm_XP_012214147.1_CoccidioidesImmitisRS,MSTSNLPLDIGTLLDLSTDQFVEDLGSSSHSSLLDQDQLDQLINFN...,242,108,"[179044.502422884, 179044.502422884, 179044.50...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2, -0.2, -0....","[0.0, 0.0, 0.4133333333333334, 0.4555555555555...","[215199.18805174268, 215199.18805174268, 21519...",108,1305
1208,Cpos_XP_003070205.1_CoccidioidesPosadasiiC735,MSTSNLPLGMVSLSASAVRLVANQRPDIGTLLDLSTDQYVEDLGSS...,260,126,"[56309.23879651313, 56309.23879651313, 56309.2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.4133333333333334, 0.4555555555555...","[6138.0, 6138.0, 6138.0, 6138.0, 6138.0, 6138....",126,1287
1209,Mory_EHA48851.1_MagnaportheOryzae70_15,MNNTSDLGLDDFTAFGGGASAFPSPAMPGVFDIASTTASTMGTVSP...,239,101,"[165405.40017749005, 165405.40017749005, 16540...","[0.0, 0.0, 0.0, -0.2, -0.2, -0.2, -0.2, -0.4, ...","[0.0, 0.0, 0.3533333333333334, 0.2333333333333...","[156903.4915015713, 156903.4915015713, 156903....",101,1312
1210,Nfis_EAW24893.1_NeosartoryaFischeriNRRL181,MSTPNIAQDMPDFFGLPSNDFGDDFELSTEPTMLSPNQIPTGLMAV...,251,109,"[148604.20780984958, 148604.20780984958, 14860...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2, -0.2, -0....","[0.0, 0.0, 0.39555555555555555, 0.453333333333...","[177960.75802414527, 177960.75802414527, 17796...",109,1304


# Get activities for overlapping Tiles

In [6]:
positional_activity_df = []

# Read all the activities
for ls in seq_data["SmoothedActivitesLoess"]:
    positional_activity_df.append(ls)

# Make the tree

In [7]:
seq_data = seq_data.reset_index()

In [8]:
# Pad the activities so that everything lines up
positional_activity_df_padded = []
for i in range(len(positional_activity_df)):
    pad_by = seq_data.loc[i, "pad_by"]
    sub_list = positional_activity_df[i]
    
    # # Case where there is no activity data
    # if sub_list == None:
    #     print("Dropping", i)
    #     seq_data = seq_data.drop(i)
    #     continue
    
    # Put -1 in from of the current data
    new_list = [-1]*pad_by + list(sub_list)
    positional_activity_df_padded.append(new_list)

In [9]:
# Making padded activity into a dataframe
activity_position_matrix = pd.DataFrame(positional_activity_df_padded)
seq_data["SpeciesName"] = seq_data["SpeciesName"].str.replace("#", "_")
activity_position_matrix.index = seq_data["SpeciesName"]
activity_position_matrix = activity_position_matrix.fillna(0)

# Write to matrix to make it easier to read in for the correct format for ete
activity_position_matrix.to_csv("position_activities.tsv", sep='\t')

In [10]:
# Read in activity data tsv as string for ete
with open("position_activities.tsv",'r') as f:
    matrix = f.readlines()
matrix = ''.join(matrix)

# Formatting required by ete to read in the heatmap data
matrix = '#' + matrix
matrix = matrix.replace("SpeciesName","Names")

## Random code that modifies the ete functionality

In [11]:
# Replacing some of ete's built in functions with our own so that we can use the colors/formatting that we want

def get_color_gradient(self,colorscheme='Reds'):
    from PyQt5 import QtGui
    import matplotlib.colors as colors
    import matplotlib.cm as cmx
    cmap0 = colors.LinearSegmentedColormap.from_list('', ['white', 'darkblue'])
    cNorm  = colors.Normalize(vmin=0, vmax=1)
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cmap0)
    color_scale = []
    for scale in np.linspace(0, 1, 201):
        # rgba = scalarMap.to_rgba(scale)
        # hex_color = '#%02x%02x%02x' % (int(rgba[0] * 255), int(rgba[1] * 255), int(rgba[2] * 255))
        # # hex_color = '#%02x%02x%02x' %scalarMap.to_rgba(scale)[:3]
        [r,g,b,a] = scalarMap.to_rgba(scale, bytes=True)
        color_scale.append( QtGui.QColor( r, g, b, a ) )

    return color_scale

# Replacing the get_color_gradient method with our custom method
ProfileFace.get_color_gradient = get_color_gradient

def draw_heatmap_profile(self):
    try:
        from numpy import isfinite as _isfinite, ceil
    except ImportError:
        pass
    else:
        isfinite = lambda n: n and _isfinite(n)

    from PyQt5.QtGui import QColor, QBrush, QPainter, QPixmap
    from PyQt5.QtCore import QRectF

    # Calculate vector
    vector = self.node.profile
    deviation = self.node.deviation
    # If no vector, skip
    if vector is None:
        return

    colors = self.get_color_gradient()

    leaves = self.node.get_leaves()

    vlength = len(vector)
    # pixels per array position
    img_height = self.height * len(leaves)
    profile_width = self.width
    profile_height= img_height

    x_alpha = float( profile_width / (len(vector)) )

    # Creates a pixmap
    self.pixmap = QPixmap(self.width, img_height)
    self.pixmap.fill(QColor("white"))
    p = QPainter(self.pixmap)

    x2 = 0
    y  = 0
    y_step = self.height
    for leaf in leaves:
        mean_vector = leaf.profile
        deviation_vector = leaf.deviation
        # Draw heatmap
        for pos in range(vlength):
            # first and second X pixel positions
            x1 = x2
            x2 = x1 + x_alpha
            dev1 = self.fit_to_scale(deviation_vector[pos])
            mean1 = self.fit_to_scale(mean_vector[pos])
            # Set heatmap color
            # if not np.isfinite(mean1):
            #     customColor = QColor("white")
            if (mean_vector[pos] == -1.0) or (math.isnan(mean1)):
                # color_index = abs(int(ceil(((self.center_v-mean1)*100)/(self.max_value-self.center_v))))
                customColor = QColor('white') # Color of the padding values
            elif mean1>self.center_v:
                color_index = abs(int(ceil(((self.center_v-mean1)*100)/(self.max_value-self.center_v))))
                customColor = colors[100 + color_index]
            elif mean1<self.center_v:
                color_index = abs(int(ceil(((self.center_v-mean1)*100)/(self.min_value-self.center_v))))
                customColor = colors[100 - color_index]
            else:
                # color_index = abs(int(ceil(((self.center_v-mean1)*100)/(self.max_value-self.center_v))))
                customColor = colors[100]

            # Fill bar with custom color
            p.fillRect(QRectF(x1, y, x_alpha, y_step), QBrush(customColor))
        y+= y_step
        x2 = 0
    p.end()

# Replacing the draw_heatmap_profile with our custom function
ProfileFace.draw_heatmap_profile = draw_heatmap_profile

# Making the tree

In [12]:
# Read in gene tree
t_base = Tree('../data/phylogenetic_info/gcn4_gene_phylogeny.treefile', format=1)

# For some reason, ClusterTree doesn't have the format option
t = ClusterTree(t_base.write(format=0),text_array=matrix)

[686] leaf names could not be mapped to the matrix rows.


In [13]:
seq_data[seq_data["SpeciesName"] == 'Saccharomycotina_jgi|Ascru1|75671|fgenesh1_kg.5___329___Locus242v1rpkm674.40']

,index,SpeciesName,Seq,Length,WxxLF_loc,SmoothedActivites,LinearCharge,LinearHydrophobicityKD,SmoothedActivitesLoess,Location_WxxLF,pad_by
22,61,Saccharomycotina_jgi|Ascru1|75671|fgenesh1_kg....,MSDFLFDSIPEFPTNSTSIVGESIFDSFAPNSLEDLNFNNPPSNHL...,387,142,"[235574.17350285183, 235574.17350285183, 23557...","[0.0, 0.0, -0.2, -0.2, -0.4, -0.2, -0.2, -0.2,...","[0.0, 0.0, 0.5933333333333334, 0.6133333333333...","[147759.68359176075, 147759.68359176075, 14775...",142,1271


In [14]:
# Find species not in the tree
set(seq_data["SpeciesName"]).difference(set(t.get_leaf_names()))

set()

In [15]:
# Lots of things in the tree that we don't have data for
set(t.get_leaf_names()).difference(set(seq_data["SpeciesName"]))

{'Blastocladiomycota_jgi|Catan2|1451137|fgenesh1_kg.199___17___Locus3917v1rpkm44.08',
 'Blastocladiomycota_jgi|Catan2|1519271|estExt_Genemark1.C_1990010',
 'Blastocladiomycota_jgi|Catan2|248926|Catan1.CE87754_8246',
 'Dothideomycetes_jgi|Boeex1|291205|gw1.12.88.1',
 'Dothideomycetes_jgi|Boeex1|295363|gw1.12.746.1',
 'Dothideomycetes_jgi|Boeex1|319687|e_gw1.12.534.1',
 'Dothideomycetes_jgi|Boeex1|320571|e_gw1.12.746.1',
 'Dothideomycetes_jgi|Boeex1|343812|estExt_Genewise1.C_12_t10429',
 'Dothideomycetes_jgi|Boeex1|343813|estExt_Genewise1.C_12_t10430',
 'Dothideomycetes_jgi|Boeex1|366995|estExt_Genewise1Plus.C_12_t10424',
 'Dothideomycetes_jgi|Boeex1|366996|estExt_Genewise1Plus.C_12_t10425',
 'Dothideomycetes_jgi|Boeex1|366997|estExt_Genewise1Plus.C_12_t10426',
 'Dothideomycetes_jgi|Boeex1|403055|fgenesh1_kg.12___411___TRINITY_DN4084_c1_g1_i1',
 'Dothideomycetes_jgi|Boeex1|403056|fgenesh1_kg.12___412___TRINITY_DN4084_c1_g2_i1',
 'Dothideomycetes_jgi|Boeex1|427973|estExt_Genemark1.C_12_t1

In [21]:
# t = ClusterTree('Phylogeny_test.tree', text_array=matrix)
length=2000
width=30

# Prune the tree to only contain the species we are interested in
leafs = t.get_leaf_names()
clade_leaves = [leaf for leaf in leafs if leaf in matrix]
# clade_leaves = [leaf for leaf in clade_leaves if leaf in yeast_clade]
t.prune(clade_leaves)

array =  t.arraytable

# Calculates some stats on the matrix. Needed to establish the color gradients.
matrix_dist = [i for r in range(len(array.matrix))\
               for i in array.matrix[r] if np.isfinite(i)]
matrix_max = np.max(matrix_dist)
matrix_min = np.min(matrix_dist)
matrix_avg = matrix_min+((matrix_max-matrix_min)/2)


def mylayout(node):
    # Creates a profile face that will represent node's profile as a
    # heatmap
    profileFace  = ProfileFace(matrix_max, 0, matrix_avg, \
                                            length, width, "heatmap", colorscheme=0)
    
    # This is the beatufil line that allows the heatmap to look good on the whole tree!
    profileFace.rotable = False
    # Creates my own layout function that uses previous faces
    # If node is a leaf
    if node.is_leaf():
        name = TextFace(node.name, fsize=15)
        add_face_to_node(name, node, column=0, position="aligned")

        # And a line profile
        add_face_to_node(profileFace, node, 1, position="aligned")
        node.img_style["size"]=0


# Use my layout to visualize the tree
ts = TreeStyle()
ts.layout_fn = mylayout

ts.mode = "c"
ts.show_leaf_name = False    
ts.draw_guiding_lines = True

# Use my layout to visualize the tree
# ts = TreeStyle()
# ts.layout_fn = mylayout
# t.show(tree_style=ts)

t.render("../figures/yeast_gene_tree_heatmaps_loess_smoothing.pdf", tree_style = ts)

{'nodes': [[5051.933513184987,
   5166.892478641935,
   5056.014385269465,
   5170.973350726412,
   0,
   None],
  [5050.531974293771,
   5185.29960515775,
   5055.191545047535,
   5189.959175911515,
   2,
   None],
  [5315.649973179438,
   5103.915308745547,
   5320.842740182009,
   5109.108075748118,
   4,
   None],
  [5380.921673145468,
   5012.807948390945,
   5386.489647988662,
   5018.375923234139,
   5,
   None],
  [5371.60114306153,
   4887.21807512985,
   5377.201716147455,
   4892.818648215775,
   6,
   None],
  [5250.55022059023,
   4799.456127969865,
   5255.363893276776,
   4804.269800656411,
   7,
   None],
  [5058.444263322106,
   4786.837419124423,
   5063.37511328958,
   4791.768269091897,
   8,
   None],
  [4791.625279816769,
   4962.32607708794,
   4797.047360483917,
   4967.748157755088,
   9,
   None],
  [4802.473754209975,
   5455.683681063062,
   4808.098876899064,
   5461.308803752151,
   10,
   None],
  [5527.92811728639,
   5596.07732702894,
   5533.5643723755